In [ ]:
# Network Scraper Results Analysis with Standardized Domain-Based Processing
# Unified evaluation system consistent with all scrapers
# Two-cell structure: Analysis + Export

import pymongo
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from urllib.parse import urlparse
from IPython.display import HTML, display
import json
import html

# --- CONFIGURATION ---
MONGO_HOST = 'mongo'
MONGO_PORT = 27017
MONGO_USER = 'admin'
MONGO_PASS = 'changeme'
MONGO_DB_NAME = 'tasks'

# Support for multiple task names - empty list means analyze ALL tasks
TASK_NAMES_TO_ANALYZE = []

def get_main_domain_from_url(url):
    """Extract main domain from URL (consistent with scrapers)"""
    try:
        parsed_url = urlparse(url)
        netloc = parsed_url.netloc
        
        subdomains_to_remove = ["sso.", "idp.", "login.", "www."]
        for subdomain in subdomains_to_remove:
            if netloc.startswith(subdomain):
                netloc = netloc[len(subdomain):]
        
        parts = netloc.split(".")
        if len(parts) > 2:
            netloc = ".".join(parts[-2:])
        
        return f"{parsed_url.scheme}://{netloc}"
    except Exception:
        return url

def detect_secure_html_indicators(url_data):
    """
    STANDARDIZED: Detect secure HTML indicators (consistent across all scrapers)
    Returns True if domain has webauthn inputs or passkey buttons
    """
    return (url_data.get('webauthn_input_found', False) or 
            url_data.get('passkey_button_found', False))

def detect_secure_specific_indicators(url_data):
    """
    STANDARDIZED: Detect secure specific indicators for network_scraper
    Returns True if domain has secure network requests or responses
    """
    secure_requests = url_data.get('secure_passkey_requests', [])
    secure_responses = url_data.get('secure_passkey_responses', [])
    return ((isinstance(secure_requests, list) and len(secure_requests) > 0) or
            (isinstance(secure_responses, list) and len(secure_responses) > 0))

def detect_possible_indicators(url_data):
    """
    STANDARDIZED: Detect possible indicators for network_scraper
    Returns True if domain has possible network patterns or requests
    """
    possible_req = url_data.get('possible_passkey_requests', [])
    possible_resp = url_data.get('possible_passkey_responses', [])
    passkey_req = url_data.get('passkey_requests', [])
    passkey_resp = url_data.get('passkey_responses', [])
    patterns = url_data.get('passkey_patterns_detected', False)
    
    return ((isinstance(possible_req, list) and len(possible_req) > 0) or
            (isinstance(possible_resp, list) and len(possible_resp) > 0) or
            (isinstance(passkey_req, list) and len(passkey_req) > 0) or
            (isinstance(passkey_resp, list) and len(passkey_resp) > 0) or
            patterns)

def detect_fedcm_indicators(url_data):
    """
    STANDARDIZED: Detect FedCM indicators for network_scraper
    Returns True if domain has FedCM network detections
    """
    fedcm_detections = url_data.get('fedcm_detections', [])
    return (isinstance(fedcm_detections, list) and len(fedcm_detections) > 0)

def evaluate_domain_classification(domain_data):
    """
    STANDARDIZED domain classification logic used across all scrapers.
    Each domain is classified into exactly ONE category with weighted scoring:
    - Secure Both (HTML + Specific): 2 points
    - Secure HTML Only: 1 point
    - Secure Specific Only: 1 point
    - No Secure Indicators: 0 points
    """
    # Check all URLs in domain for any indicators
    has_secure_specific = False
    has_secure_html = False
    has_possible = False
    has_fedcm = False
    
    for url_data in domain_data:
        if detect_secure_specific_indicators(url_data):
            has_secure_specific = True
        if detect_secure_html_indicators(url_data):
            has_secure_html = True
        if detect_possible_indicators(url_data):
            has_possible = True
        if detect_fedcm_indicators(url_data):
            has_fedcm = True
    
    # Classify domain based on secure indicators
    classification = {
        'secure_html_only': False,
        'secure_specific_only': False,
        'secure_both': False,
        'no_secure_found': False,
        'has_possible': has_possible,
        'has_fedcm': has_fedcm,
        'weighted_score': 0
    }
    
    if has_secure_html and has_secure_specific:
        classification['secure_both'] = True
        classification['weighted_score'] = 2
    elif has_secure_html:
        classification['secure_html_only'] = True
        classification['weighted_score'] = 1
    elif has_secure_specific:
        classification['secure_specific_only'] = True
        classification['weighted_score'] = 1
    else:
        classification['no_secure_found'] = True
        classification['weighted_score'] = 0
    
    return classification

def calculate_indicators_per_task(df, domain_groups):
    """Calculate indicators per task_name for task-specific visualization"""
    task_indicators = {}
    
    if 'task_name' in df.columns:
        for task_name in df['task_name'].unique():
            # Get domains that belong to this task
            task_df = df[df['task_name'] == task_name]
            task_domain_groups = defaultdict(list)
            
            for _, row in task_df.iterrows():
                domain = row.get('url_id', 'unknown')
                task_domain_groups[domain].append(row)
            
            # Calculate indicators for this task's domains
            secure_html_only = 0
            secure_specific_only = 0
            secure_both = 0
            possible_count = 0
            fedcm_count = 0
            
            for domain, domain_data in task_domain_groups.items():
                classification = evaluate_domain_classification(domain_data)
                
                if classification['secure_html_only']:
                    secure_html_only += 1
                elif classification['secure_specific_only']:
                    secure_specific_only += 1
                elif classification['secure_both']:
                    secure_both += 1
                    
                if classification['has_possible']:
                    possible_count += 1
                if classification['has_fedcm']:
                    fedcm_count += 1
            
            task_indicators[task_name] = {
                'secure_html_only': secure_html_only,
                'secure_specific_only': secure_specific_only,
                'secure_both': secure_both,
                'possible_passkey': possible_count,
                'fedcm': fedcm_count,
                'total_domains': len(task_domain_groups)
            }
    
    return task_indicators

def load_and_process_data(db, collection_name, task_names=None):
    """Load data from MongoDB and process it for domain-based analysis"""
    collection = db[collection_name]
    
    # Build query for specific task names if provided - USE task_name at document level
    query = {}
    if task_names:
        query = {"task_name": {"$in": task_names}}
    
    # Load all data for processing
    data = list(collection.find(query))
    if not data:
        print(f"No documents found in '{collection_name}' for tasks: {task_names}")
        return pd.DataFrame(), {}

    normalized_data = []
    for doc in data:
        if 'result' in doc and doc['result'] is not None:
            if isinstance(doc['result'], dict):
                flat_doc = pd.json_normalize(doc['result']).to_dict(orient='records')[0]
                
                # FIXED: Get task_name from document level, not from result
                if 'task_name' in doc:
                    flat_doc['task_name'] = doc['task_name']
                elif 'task_name' not in flat_doc:
                    flat_doc['task_name'] = 'unknown'
                
                # Handle timeout and duration
                if 'task_config.timeout' in flat_doc:
                    flat_doc['timeout'] = flat_doc['task_config.timeout']
                elif 'timeout' not in flat_doc:
                    flat_doc['timeout'] = 60
                    
                if 'duration_seconds' not in flat_doc:
                    flat_doc['duration_seconds'] = np.nan
                
                # Ensure url_id from result
                if 'url_id' not in flat_doc and 'url' in flat_doc:
                    try:
                        flat_doc['url_id'] = get_main_domain_from_url(flat_doc['url'])
                    except:
                        flat_doc['url_id'] = 'unknown'
                    
                # Add default fields if missing
                for field in ['secure_passkey_requests', 'secure_passkey_responses', 
                             'possible_passkey_requests', 'possible_passkey_responses',
                             'passkey_requests', 'passkey_responses', 'fedcm_detections']:
                    if field not in flat_doc:
                        flat_doc[field] = []
                        
                if 'passkey_patterns_detected' not in flat_doc:
                    flat_doc['passkey_patterns_detected'] = False
                    
                # Handle NaN values in boolean columns
                for bool_col in ['passkey_patterns_detected', 'error', 'passkey_button_found', 'webauthn_input_found']:
                    if bool_col in flat_doc and pd.isna(flat_doc[bool_col]):
                        flat_doc[bool_col] = False
                
                if 'error_messages' not in flat_doc:
                    flat_doc['error_messages'] = []
                        
                normalized_data.append(flat_doc)
    
    df = pd.DataFrame(normalized_data)
    print(f"{len(df)} documents loaded from '{collection_name}' for tasks: {task_names}")
    
    if not df.empty and 'task_name' in df.columns:
        found_tasks = df['task_name'].unique()
        print(f"Tasks found in data: {', '.join(found_tasks)}")
    
    # Group by domain (url_id) for domain-based processing
    domain_groups = defaultdict(list)
    for _, row in df.iterrows():
        domain = row.get('url_id', 'unknown')
        domain_groups[domain].append(row)
    
    return df, domain_groups

def calculate_standardized_metrics(df, domain_groups):
    """Calculate metrics using standardized domain classification"""
    total_domains = len(domain_groups)
    
    # Counters using standardized classification
    secure_html_only = 0
    secure_specific_only = 0
    secure_both = 0
    possible_count = 0
    fedcm_count = 0
    weighted_total = 0
    error_count = 0
    
    for domain, domain_data in domain_groups.items():
        classification = evaluate_domain_classification(domain_data)
        
        if classification['secure_html_only']:
            secure_html_only += 1
        elif classification['secure_specific_only']:
            secure_specific_only += 1
        elif classification['secure_both']:
            secure_both += 1
            
        if classification['has_possible']:
            possible_count += 1
        if classification['has_fedcm']:
            fedcm_count += 1
            
        weighted_total += classification['weighted_score']
        
        # Check for errors in any URL of this domain
        for url_data in domain_data:
            if url_data.get('error', False):
                error_count += 1
                break
    
    total_secure = secure_html_only + secure_specific_only + secure_both
    
    # Timeout analysis for all URLs
    timeout_analysis = analyze_timeouts(df)
    
    # Calculate per-task indicators
    task_indicators = calculate_indicators_per_task(df, domain_groups)
    
    metrics = {
        'total_domains': total_domains,
        'total_urls': len(df),
        'secure_html_only': secure_html_only,
        'secure_specific_only': secure_specific_only,
        'secure_both': secure_both,
        'total_secure': total_secure,
        'weighted_total': weighted_total,
        'possible_passkey': possible_count,
        'fedcm': fedcm_count,
        'errors': error_count,
        'task_indicators': task_indicators
    }
    
    return metrics, timeout_analysis

def analyze_timeouts(df):
    """Analyze timeout compliance for all URLs per task"""
    timeout_data = []
    
    if 'task_name' in df.columns:
        for task_name in df['task_name'].unique():
            task_df = df[df['task_name'] == task_name]
            if not task_df.empty and 'timeout' in task_df.columns and 'duration_seconds' in task_df.columns:
                timeout_val = task_df['timeout'].iloc[0]
                if pd.notna(timeout_val) and timeout_val > 0:
                    valid_durations = task_df.dropna(subset=['duration_seconds'])
                    if not valid_durations.empty:
                        compliance_ratios = valid_durations['duration_seconds'] / timeout_val
                        
                        counts = {
                            '0-50%': int((compliance_ratios <= 0.5).sum()),
                            '50-100%': int(((compliance_ratios > 0.5) & (compliance_ratios <= 1.0)).sum()),
                            '100-150%': int(((compliance_ratios > 1.0) & (compliance_ratios <= 1.5)).sum()),
                            '>150%': int((compliance_ratios > 1.5).sum())
                        }
                        
                        timeout_data.append({
                            'task_name': task_name,
                            'timeout': int(timeout_val),
                            'total_urls': len(valid_durations),
                            **counts
                        })
    
    return timeout_data

def create_standardized_visualization(metrics, timeout_analysis, domain_groups):
    """Create standardized visualization with 6 plots - task indicators spanning full width at the end"""
    fig = plt.figure(figsize=(16, 22))
    fig.patch.set_facecolor('#fafafa')
    fig.suptitle('Network Scraper - Standardized Domain-Based Analysis', 
                fontsize=18, fontweight='bold', color='#2d1b69', y=0.99)
    
    # Create subplot layout using subplot2grid (compatible with tight_layout)
    ax1 = plt.subplot2grid((4, 2), (0, 0))
    ax2 = plt.subplot2grid((4, 2), (0, 1))
    ax3 = plt.subplot2grid((4, 2), (1, 0))
    ax4 = plt.subplot2grid((4, 2), (1, 1))
    ax5 = plt.subplot2grid((4, 2), (2, 0), colspan=2)
    ax6 = plt.subplot2grid((4, 2), (3, 0), colspan=2)
    
    # Modern colors matching comparison
    colors = {
        'primary': '#3498db',      # HTML only
        'secondary': '#2ecc71',    # Specific only  
        'both': '#9b59b6',        # Both indicators
        'possible': '#f39c12',     # Possible indicators
        'fedcm': '#3742fa',        # FedCM indicators
        'success': '#20bf6b',      # Success metrics
        'warning': '#ff6b6b'       # Warnings/errors
    }
    
    # 1. Domain vs URL Count
    counts = [metrics['total_domains'], metrics['total_urls']]
    bars1 = ax1.bar(['Unique Domains', 'Total URLs'], counts, 
                   color=[colors['primary'], colors['secondary']], alpha=0.8, 
                   edgecolor='white', linewidth=2)
    ax1.set_title('Domains vs URLs', fontsize=14, fontweight='bold', color='#2d1b69',pad=20)
    ax1.set_ylabel('Count', fontsize=12, color='#374151')
    
    for bar, count in zip(bars1, counts):
        percentage = (count / sum(counts)) * 100
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(counts)*0.01, 
                f'{count} ({percentage:.1f}%)', ha='center', va='bottom', 
                fontweight='bold', fontsize=11, color='#2d1b69')
    
    ax1.spines['top'].set_visible(False)
    ax1.spines['right'].set_visible(False)
    ax1.grid(axis='y', alpha=0.3)
    
    # 2. Standardized Indicator Classification with Stacked Bars
    categories = ['Secure Indicators', 'Possible', 'FedCM']
    
    # Create stacked bar for secure indicators
    secure_html_data = [metrics['secure_html_only'], 0, 0]
    secure_specific_data = [metrics['secure_specific_only'], 0, 0] 
    secure_both_data = [metrics['secure_both'], 0, 0]
    possible_data = [0, metrics['possible_passkey'], 0]
    fedcm_data = [0, 0, metrics['fedcm']]
    
    # Stacked bars for secure indicators
    bars2_html = ax2.bar(categories, secure_html_data,
                        color=colors['primary'], alpha=0.8, edgecolor='white', linewidth=1,
                        label='HTML Only (1pt)')
    bars2_specific = ax2.bar(categories, secure_specific_data, bottom=secure_html_data,
                           color=colors['secondary'], alpha=0.8, edgecolor='white', linewidth=1,
                           label='Specific Only (1pt)')
    bars2_both = ax2.bar(categories, secure_both_data,
                        bottom=[h + s for h, s in zip(secure_html_data, secure_specific_data)],
                        color=colors['both'], alpha=0.8, edgecolor='white', linewidth=1,
                        label='Both (2pts)')
    
    # Separate bars for possible and fedcm
    bars2_possible = ax2.bar(categories, possible_data,
                           color=colors['possible'], alpha=0.8, edgecolor='white', linewidth=1,
                           label='Possible')
    bars2_fedcm = ax2.bar(categories, fedcm_data,
                         color=colors['fedcm'], alpha=0.8, edgecolor='white', linewidth=1,
                         label='FedCM')
    
    # Add numbers INSIDE the bar segments for secure indicators (first category only)
    if secure_html_data[0] > 0:  # HTML only count
        ax2.text(0, secure_html_data[0]/2, str(secure_html_data[0]), 
                ha='center', va='center', fontweight='bold', fontsize=11, color='white')
    
    if secure_specific_data[0] > 0:  # Specific only count
        y_pos = secure_html_data[0] + secure_specific_data[0]/2
        ax2.text(0, y_pos, str(secure_specific_data[0]), 
                ha='center', va='center', fontweight='bold', fontsize=11, color='white')
    
    if secure_both_data[0] > 0:  # Both indicators count
        y_pos = secure_html_data[0] + secure_specific_data[0] + secure_both_data[0]/2
        ax2.text(0, y_pos, str(secure_both_data[0]), 
                ha='center', va='center', fontweight='bold', fontsize=11, color='white')
    
    # Add numbers inside other bars if they have values
    if possible_data[1] > 0:  # Possible indicators
        ax2.text(1, possible_data[1]/2, str(possible_data[1]), 
                ha='center', va='center', fontweight='bold', fontsize=11, color='white')
    
    if fedcm_data[2] > 0:  # FedCM indicators
        ax2.text(2, fedcm_data[2]/2, str(fedcm_data[2]), 
                ha='center', va='center', fontweight='bold', fontsize=11, color='white')
    
    ax2.set_title('Standardized Indicator Classification (Domain-Based)', fontsize=14, fontweight='bold', color='#2d1b69',pad=20)
    ax2.set_ylabel('Number of Domains', fontsize=12, color='#374151')
    
    # Add value labels
    total_secure = metrics['secure_html_only'] + metrics['secure_specific_only'] + metrics['secure_both']
    bar_totals = [total_secure, metrics['possible_passkey'], metrics['fedcm']]
    
    for i, (category, total) in enumerate(zip(categories, bar_totals)):
        if total > 0:
            percentage = (total / metrics['total_domains']) * 100 if metrics['total_domains'] > 0 else 0
            label_text = f'{total} ({percentage:.1f}%)'
            if i == 0:  # Secure indicators - add weighted score
                label_text += f'\n{metrics["weighted_total"]} pts'
            ax2.text(i, total + max(bar_totals)*0.01, label_text, 
                    ha='center', va='bottom', fontweight='bold', fontsize=10, color='#2d1b69')
    
    ax2.legend(loc='upper right', frameon=True, fontsize=9)
    ax2.spines['top'].set_visible(False) 
    ax2.spines['right'].set_visible(False)
    ax2.grid(axis='y', alpha=0.3)
    
    # 3. Detection Overview Pie Chart with Weighted Scores
    if metrics['total_secure'] > 0 or metrics['possible_passkey'] > 0:
        pie_labels = []
        pie_values = []
        pie_colors = []
        
        if metrics['secure_html_only'] > 0:
            pie_labels.append(f'HTML Only\n({metrics["secure_html_only"]} domains, 1pt each)')
            pie_values.append(metrics['secure_html_only'])
            pie_colors.append(colors['primary'])
            
        if metrics['secure_specific_only'] > 0:
            pie_labels.append(f'Specific Only\n({metrics["secure_specific_only"]} domains, 1pt each)')
            pie_values.append(metrics['secure_specific_only'])
            pie_colors.append(colors['secondary'])
            
        if metrics['secure_both'] > 0:
            pie_labels.append(f'Both Indicators\n({metrics["secure_both"]} domains, 2pts each)')
            pie_values.append(metrics['secure_both'])
            pie_colors.append(colors['both'])
            
        if metrics['possible_passkey'] > 0:
            pie_labels.append(f'Possible Only\n({metrics["possible_passkey"]} domains)')
            pie_values.append(metrics['possible_passkey'])
            pie_colors.append(colors['possible'])
            
        no_indicators = metrics['total_domains'] - total_secure - metrics['possible_passkey']
        if no_indicators > 0:
            pie_labels.append(f'No Indicators\n({no_indicators} domains)')
            pie_values.append(no_indicators)
            pie_colors.append('#6c757d')
        
        wedges, texts, autotexts = ax3.pie(pie_values, labels=pie_labels,
                                          autopct='%1.1f%%', startangle=90,
                                          colors=pie_colors, textprops={'fontsize': 9},
                                          wedgeprops={'edgecolor': 'white', 'linewidth': 2})
        
        ax3.set_title(f'Domain Classification Overview\nWeighted Score: {metrics["weighted_total"]} points', 
                     fontsize=14, fontweight='bold', color='#2d1b69',pad=20)
    else:
        ax3.text(0.5, 0.5, 'No indicators found', ha='center', va='center',
                transform=ax3.transAxes, fontsize=14, color='#6b7280')
        ax3.set_title('Domain Classification Overview', fontsize=14, fontweight='bold', color='#2d1b69',pad=20)
    
    # 4. Timeout Compliance
    if timeout_analysis:
        timeout_df = pd.DataFrame(timeout_analysis)
        tasks = timeout_df['task_name'].tolist()
        
        categories = ['0-50%', '50-100%', '100-150%', '>150%']
        colors_timeout = ['#90EE90', '#28a745', '#ffc107', '#dc3545']
        
        bottom = np.zeros(len(tasks))
        for i, category in enumerate(categories):
            values = timeout_df[category].tolist()
            bars = ax4.bar(tasks, values, bottom=bottom, label=category,
                          color=colors_timeout[i], alpha=0.8, edgecolor='white', linewidth=1)
            
            # Add absolute numbers on bars
            for j, (bar, value) in enumerate(zip(bars, values)):
                if value > 0:
                    ax4.text(bar.get_x() + bar.get_width()/2, bar.get_y() + bar.get_height()/2,
                            str(value), ha='center', va='center', fontweight='bold',
                            fontsize=9, color='white' if i in [1, 3] else 'black')
            
            bottom += values
        
        ax4.set_title('Timeout Compliance (All URLs per Task)', fontsize=14, fontweight='bold', color='#2d1b69',pad=20)
        ax4.set_xlabel('Task', fontsize=12, color='#374151')
        ax4.set_ylabel('Number of URLs', fontsize=12, color='#374151')
        ax4.legend(frameon=True, fontsize=10)
        ax4.tick_params(axis='x', rotation=45, colors='#374151')
        ax4.spines['top'].set_visible(False)
        ax4.spines['right'].set_visible(False)
        ax4.grid(axis='y', alpha=0.3)
    else:
        ax4.text(0.5, 0.5, 'No timeout data available', ha='center', va='center',
                transform=ax4.transAxes, fontsize=14, color='#6b7280')
        ax4.set_title('Timeout Analysis', fontsize=14, fontweight='bold', color='#2d1b69',pad=20)
    
    # 5. Domain Classification by Indicators and Errors (spanning full width)
    indicator_error_data = classify_domains_by_indicators_and_errors(domain_groups)
    
    categories = ['Secure + No Error', 'Secure + Error', 'No Secure + No Error', 'No Secure + Error']
    counts = [
        indicator_error_data['secure_no_error'],
        indicator_error_data['secure_with_error'], 
        indicator_error_data['no_secure_no_error'],
        indicator_error_data['no_secure_with_error']
    ]
    
    colors_bars = ['#20bf6b', '#f39c12', '#6c757d', '#ff6b6b']
    bars5 = ax5.bar(categories, counts, color=colors_bars, alpha=0.8, edgecolor='white', linewidth=2)
    
    ax5.set_title('Domains by Indicators and Error Status', fontsize=14, fontweight='bold', color='#2d1b69',pad=20)
    ax5.set_ylabel('Number of Domains', fontsize=12, color='#374151')
    ax5.set_xlabel('Domain Category', fontsize=12, color='#374151')
    
    # Add value labels on bars
    max_count = max(counts) if counts else 1
    for bar, count in zip(bars5, counts):
        percentage = (count / sum(counts)) * 100 if sum(counts) > 0 else 0
        ax5.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max_count*0.01,
                f'{count}\n({percentage:.1f}%)', ha='center', va='bottom', 
                fontweight='bold', fontsize=10, color='#2d1b69')
    
    ax5.tick_params(axis='x', rotation=45, colors='#374151')
    ax5.spines['top'].set_visible(False)
    ax5.spines['right'].set_visible(False)
    ax5.grid(axis='y', alpha=0.3)
    
    # 6. Task-specific Indicators (spanning full width)
    task_indicators = metrics.get('task_indicators', {})
    if task_indicators:
        task_names = list(task_indicators.keys())
        
        # Set up bar positions
        x = np.arange(len(task_names))
        width = 0.25
        
        # Data for each indicator type
        secure_html_data = [task_indicators[task]['secure_html_only'] for task in task_names]
        secure_specific_data = [task_indicators[task]['secure_specific_only'] for task in task_names]
        secure_both_data = [task_indicators[task]['secure_both'] for task in task_names]
        possible_data = [task_indicators[task]['possible_passkey'] for task in task_names]
        fedcm_data = [task_indicators[task]['fedcm'] for task in task_names]
        
        # Create grouped bars for each category
        # Secure indicators (stacked)
        bars_html = ax6.bar(x - width, secure_html_data, width, 
                          color=colors['primary'], alpha=0.8, edgecolor='white', linewidth=1,
                          label='HTML Only (1pt)')
        bars_specific = ax6.bar(x - width, secure_specific_data, width, bottom=secure_html_data,
                              color=colors['secondary'], alpha=0.8, edgecolor='white', linewidth=1,
                              label='Specific Only (1pt)')
        bars_both = ax6.bar(x - width, secure_both_data, width, 
                          bottom=[h + s for h, s in zip(secure_html_data, secure_specific_data)],
                          color=colors['both'], alpha=0.8, edgecolor='white', linewidth=1,
                          label='Both (2pts)')
        
        # Possible indicators
        bars_possible = ax6.bar(x, possible_data, width,
                              color=colors['possible'], alpha=0.8, edgecolor='white', linewidth=1,
                              label='Possible')
        
        # FedCM indicators
        bars_fedcm = ax6.bar(x + width, fedcm_data, width,
                           color=colors['fedcm'], alpha=0.8, edgecolor='white', linewidth=1,
                           label='FedCM')
        
        # Add labels for secure stacked bars
        for i, task in enumerate(task_names):
            x_pos = i - width
            total_secure = (secure_html_data[i] + secure_specific_data[i] + secure_both_data[i])
            if total_secure > 0:
                # Calculate weighted score for this task
                weighted_score = (secure_html_data[i] + secure_specific_data[i] + secure_both_data[i] * 2)
                ax6.text(x_pos, total_secure + 0.1, f'{total_secure}\n({weighted_score}pts)', 
                       ha='center', va='bottom', fontweight='bold', fontsize=9, color='#2d1b69')
        
        # Add labels for possible and fedcm
        def add_value_labels_ax6(bars, values):
            for bar, value in zip(bars, values):
                if value > 0:
                    ax6.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.1,
                           f'{value}', ha='center', va='bottom', fontweight='bold', fontsize=9)
        
        add_value_labels_ax6(bars_possible, possible_data)
        add_value_labels_ax6(bars_fedcm, fedcm_data)
        
        # Customize the plot
        ax6.set_title('Indicator Classification per Task Name (Domain-Based)', 
                    fontsize=14, fontweight='bold', color='#2d1b69', pad=20)
        ax6.set_xlabel('Task Name', fontsize=12, color='#374151')
        ax6.set_ylabel('Number of Domains', fontsize=12, color='#374151')
        ax6.set_xticks(x)
        ax6.set_xticklabels(task_names, rotation=45, ha='right', color='#374151')
        
        # Add legend
        ax6.legend(loc='upper right', frameon=True, fontsize=10)
        
        # Style the plot
        ax6.spines['top'].set_visible(False)
        ax6.spines['right'].set_visible(False)
        ax6.grid(axis='y', alpha=0.3)
    else:
        ax6.text(0.5, 0.5, 'No task indicators data available', ha='center', va='center',
                transform=ax6.transAxes, fontsize=14, color='#6b7280')
        ax6.set_title('Task-specific Indicator Analysis', fontsize=14, fontweight='bold', color='#2d1b69',pad=20)
    
    plt.tight_layout()
    plt.show()
    
    return fig

def classify_domains_by_indicators_and_errors(domain_groups):
    """Classify domains by secure indicators and error status"""
    secure_no_error = 0
    secure_with_error = 0
    no_secure_no_error = 0
    no_secure_with_error = 0
    error_details = {}
    
    for domain, domain_data in domain_groups.items():
        # Check if domain has secure indicators
        classification = evaluate_domain_classification(domain_data)
        has_secure = not classification['no_secure_found']
        
        # Check if domain has errors
        has_error = False
        for url_data in domain_data:
            if url_data.get('error', False):
                has_error = True
                # Collect error types
                error_msg = url_data.get('error_messages', [])
                if isinstance(error_msg, list) and error_msg:
                    for msg in error_msg:
                        if isinstance(msg, str):
                            error_type = msg.split(':')[0] if ':' in msg else 'Other'
                            error_details[error_type] = error_details.get(error_type, 0) + 1
                elif isinstance(error_msg, str) and error_msg:
                    error_type = error_msg.split(':')[0] if ':' in error_msg else 'Other'
                    error_details[error_type] = error_details.get(error_type, 0) + 1
                else:
                    error_details['Unknown'] = error_details.get('Unknown', 0) + 1
                break
        
        # Classify domain
        if has_secure and not has_error:
            secure_no_error += 1
        elif has_secure and has_error:
            secure_with_error += 1
        elif not has_secure and not has_error:
            no_secure_no_error += 1
        else:  # not has_secure and has_error
            no_secure_with_error += 1
    
    return {
        'secure_no_error': secure_no_error,
        'secure_with_error': secure_with_error,
        'no_secure_no_error': no_secure_no_error,
        'no_secure_with_error': no_secure_with_error,
        'error_details': error_details
    }

def create_summary_table(metrics):
    """Create summary table matching comparison format"""
    table_html = '''<h3>📊 Network Scraper - Standardized Domain Results</h3>
    <table style="border-collapse: collapse; width: 100%; margin: 20px 0;">
        <tr style="background-color: #f2f2f2;">
            <th style="border: 1px solid #ddd; padding: 12px; text-align: left;">Metric</th>
            <th style="border: 1px solid #ddd; padding: 12px; text-align: center;">Count</th>
            <th style="border: 1px solid #ddd; padding: 12px; text-align: center;">Percentage</th>
            <th style="border: 1px solid #ddd; padding: 12px; text-align: center;">Weighted Points</th>
        </tr>'''
    
    total_domains = metrics['total_domains']
    
    rows = [
        ('Total Domains', metrics['total_domains'], 100.0, '-'),
        ('Total URLs', metrics['total_urls'], '-', '-'),
        ('Secure HTML Only', metrics['secure_html_only'], 
         (metrics['secure_html_only']/total_domains*100) if total_domains > 0 else 0, 
         f"{metrics['secure_html_only']} × 1"),
        ('Secure Specific Only', metrics['secure_specific_only'], 
         (metrics['secure_specific_only']/total_domains*100) if total_domains > 0 else 0,
         f"{metrics['secure_specific_only']} × 1"),
        ('Secure Both', metrics['secure_both'], 
         (metrics['secure_both']/total_domains*100) if total_domains > 0 else 0,
         f"{metrics['secure_both']} × 2"),
        ('Total Secure', metrics['total_secure'], 
         (metrics['total_secure']/total_domains*100) if total_domains > 0 else 0,
         f"{metrics['weighted_total']} pts"),
        ('Possible Indicators', metrics['possible_passkey'], 
         (metrics['possible_passkey']/total_domains*100) if total_domains > 0 else 0, '-'),
        ('FedCM Indicators', metrics['fedcm'], 
         (metrics['fedcm']/total_domains*100) if total_domains > 0 else 0, '-'),
        ('Errors', metrics['errors'], 
         (metrics['errors']/total_domains*100) if total_domains > 0 else 0, '-')
    ]
    
    for metric, count, percentage, points in rows:
        perc_str = f"{percentage:.1f}%" if isinstance(percentage, (int, float)) and percentage != '-' else str(percentage)
        table_html += f'''
        <tr>
            <td style="border: 1px solid #ddd; padding: 12px; font-weight: bold;">{metric}</td>
            <td style="border: 1px solid #ddd; padding: 12px; text-align: center;">{count}</td>
            <td style="border: 1px solid #ddd; padding: 12px; text-align: center;">{perc_str}</td>
            <td style="border: 1px solid #ddd; padding: 12px; text-align: center; color: purple; font-weight: bold;">{points}</td>
        </tr>'''
    
    table_html += '''</table>
    <p><strong>Standardized Classification:</strong></p>
    <ul>
        <li><strong>Secure HTML:</strong> webauthn_input_found OR passkey_button_found</li>
        <li><strong>Secure Specific:</strong> secure network requests/responses with passkey patterns</li>
        <li><strong>Weighted Scoring:</strong> HTML only OR Specific only = 1 point, Both = 2 points</li>
        <li><strong>Domain Priority:</strong> Both > HTML only > Specific only > No secure indicators</li>
    </ul>'''
    
    return table_html

# ====================== MAIN EXECUTION ======================

try:
    # Connect to MongoDB
    print("🔄 Starting MongoDB data extraction...")
    client = pymongo.MongoClient(f'mongodb://{MONGO_USER}:{MONGO_PASS}@{MONGO_HOST}:{MONGO_PORT}/')
    db = client[MONGO_DB_NAME]
    
    print("📡 Loading data from network_scraper collection...")
    print("🎯 Using STANDARDIZED evaluation logic:")
    print("   • Secure HTML: webauthn_input_found OR passkey_button_found")
    print("   • Secure Specific: secure network requests/responses with passkey patterns")
    print("   • Classification: HTML only (1pt) | Specific only (1pt) | Both (2pts) | None (0pts)")
    
    df, domain_groups = load_and_process_data(db, 'network_scraper', TASK_NAMES_TO_ANALYZE)
    
    if not df.empty:
        print(f"\n✅ Data extraction complete! Found {len(domain_groups)} unique domains.")
        print(f"📊 Data Overview: {len(df)} URLs across {len(domain_groups)} domains")
        
        print("🔄 Calculating standardized metrics...")
        metrics, timeout_analysis = calculate_standardized_metrics(df, domain_groups)
        
        print(f"\n🎯 Standardized Results:")
        print(f"   • HTML only: {metrics['secure_html_only']} domains (1pt each)")
        print(f"   • Specific only: {metrics['secure_specific_only']} domains (1pt each)")
        print(f"   • Both indicators: {metrics['secure_both']} domains (2pts each)")
        print(f"   • Total secure: {metrics['total_secure']} domains")
        print(f"   • Weighted score: {metrics['weighted_total']} points")
        print(f"   • Possible: {metrics['possible_passkey']}, FedCM: {metrics['fedcm']}")
        
        print("📈 Creating standardized visualizations...")
        main_figure = create_standardized_visualization(metrics, timeout_analysis, domain_groups)
        
        print("📋 Creating summary table...")
        summary_html = create_summary_table(metrics)
        display(HTML(summary_html))
        
        # Store variables for export
        visualizations = [main_figure]
        
        print("\n✅ Standardized analysis complete!")
        print("   🎯 Network scraper now uses consistent evaluation with other scrapers")
        print("   📊 Domain-based classification with weighted scoring")
        print("   📈 Unified visualization style with task-specific analysis")
        print("   🔽 Run the next cell to save results and visualizations")
    else:
        print("❌ No data found in the network_scraper collection for the specified task names.")

except Exception as e:
    print(f"❌ Error: {e}")
    print("Please check if MongoDB is running and accessible.")
    raise

In [ ]:
# Export Results and Visualizations
# Standardized export consistent with all scrapers

import os
import json
from datetime import datetime
import matplotlib.pyplot as plt

# Export configuration
EXPORT_DIR = os.path.expanduser('~/jupyter-exports')
os.makedirs(EXPORT_DIR, exist_ok=True)

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
scraper_name = 'network_scraper'

try:
    # 1. Export raw data as CSV
    if not df.empty:
        csv_filename = f'{scraper_name}_standardized_data_{timestamp}.csv'
        csv_path = os.path.join(EXPORT_DIR, csv_filename)
        df.to_csv(csv_path, index=False)
        print(f"📄 Raw data exported: {csv_path}")
    
    # 2. Export domain classification details
    if domain_groups:
        domain_results = []
        for domain, domain_data in domain_groups.items():
            classification = evaluate_domain_classification(domain_data)
            
            # Collect indicator details
            indicators = {
                'secure_html_found': False,
                'secure_network_found': False,
                'possible_found': False,
                'fedcm_found': False,
                'indicator_counts': {
                    'webauthn_inputs': 0,
                    'passkey_buttons': 0,
                    'secure_requests': 0,
                    'secure_responses': 0,
                    'possible_requests': 0,
                    'possible_responses': 0,
                    'passkey_requests': 0,
                    'passkey_responses': 0,
                    'fedcm_detections': 0,
                    'pattern_matches': 0
                }
            }
            
            for url_data in domain_data:
                if detect_secure_html_indicators(url_data):
                    indicators['secure_html_found'] = True
                    if url_data.get('webauthn_input_found', False):
                        indicators['indicator_counts']['webauthn_inputs'] += 1
                    if url_data.get('passkey_button_found', False):
                        indicators['indicator_counts']['passkey_buttons'] += 1
                
                if detect_secure_specific_indicators(url_data):
                    indicators['secure_network_found'] = True
                    secure_req = url_data.get('secure_passkey_requests', [])
                    secure_resp = url_data.get('secure_passkey_responses', [])
                    if isinstance(secure_req, list):
                        indicators['indicator_counts']['secure_requests'] += len(secure_req)
                    if isinstance(secure_resp, list):
                        indicators['indicator_counts']['secure_responses'] += len(secure_resp)
                
                if detect_possible_indicators(url_data):
                    indicators['possible_found'] = True
                    poss_req = url_data.get('possible_passkey_requests', [])
                    poss_resp = url_data.get('possible_passkey_responses', [])
                    passkey_req = url_data.get('passkey_requests', [])
                    passkey_resp = url_data.get('passkey_responses', [])
                    if isinstance(poss_req, list):
                        indicators['indicator_counts']['possible_requests'] += len(poss_req)
                    if isinstance(poss_resp, list):
                        indicators['indicator_counts']['possible_responses'] += len(poss_resp)
                    if isinstance(passkey_req, list):
                        indicators['indicator_counts']['passkey_requests'] += len(passkey_req)
                    if isinstance(passkey_resp, list):
                        indicators['indicator_counts']['passkey_responses'] += len(passkey_resp)
                    if url_data.get('passkey_patterns_detected', False):
                        indicators['indicator_counts']['pattern_matches'] += 1
                
                if detect_fedcm_indicators(url_data):
                    indicators['fedcm_found'] = True
                    fedcm_det = url_data.get('fedcm_detections', [])
                    if isinstance(fedcm_det, list):
                        indicators['indicator_counts']['fedcm_detections'] += len(fedcm_det)
            
            domain_results.append({
                'domain': domain,
                'url_count': len(domain_data),
                'classification': {
                    'secure_html_only': classification['secure_html_only'],
                    'secure_network_only': classification['secure_specific_only'],
                    'secure_both': classification['secure_both'],
                    'no_secure': classification['no_secure_found'],
                    'has_possible': classification['has_possible'],
                    'has_fedcm': classification['has_fedcm'],
                    'weighted_score': classification['weighted_score']
                },
                'indicators': indicators
            })
        
        domain_filename = f'{scraper_name}_domain_analysis_{timestamp}.json'
        domain_path = os.path.join(EXPORT_DIR, domain_filename)
        with open(domain_path, 'w', encoding='utf-8') as f:
            json.dump(domain_results, f, indent=2, ensure_ascii=False)
        print(f"📊 Domain analysis exported: {domain_path}")
    
    # 3. Export standardized metrics summary
    if 'metrics' in locals():
        metrics_summary = {
            'scraper_type': scraper_name,
            'analysis_timestamp': timestamp,
            'standardized_metrics': metrics,
            'timeout_analysis': timeout_analysis if timeout_analysis else [],
            'export_info': {
                'total_domains_analyzed': len(domain_groups) if domain_groups else 0,
                'total_urls_analyzed': len(df) if not df.empty else 0,
                'classification_method': 'standardized_domain_based',
                'weighted_scoring': True
            }
        }
        
        metrics_filename = f'{scraper_name}_metrics_summary_{timestamp}.json'
        metrics_path = os.path.join(EXPORT_DIR, metrics_filename)
        with open(metrics_path, 'w', encoding='utf-8') as f:
            json.dump(metrics_summary, f, indent=2, ensure_ascii=False)
        print(f"📈 Metrics summary exported: {metrics_path}")
    
    # 4. Export visualizations
    if 'visualizations' in locals() and visualizations:
        for i, fig in enumerate(visualizations):
            if fig:
                viz_filename = f'{scraper_name}_standardized_analysis_{timestamp}.png'
                viz_path = os.path.join(EXPORT_DIR, viz_filename)
                fig.savefig(viz_path, dpi=300, bbox_inches='tight', facecolor='white')
                print(f"🖼️  Visualization exported: {viz_path}")
    
    # 5. Print export summary
    print("\n" + "="*80)
    print(f"✅ NETWORK SCRAPER - STANDARDIZED EXPORT COMPLETE")
    print("="*80)
    print(f"📂 Export Directory: {EXPORT_DIR}")
    print(f"⏰ Timestamp: {timestamp}")
    print()
    print(f"📊 Standardized Domain Analysis:")
    if 'metrics' in locals():
        print(f"   • Total domains: {metrics['total_domains']}")
        print(f"   • HTML only (1pt): {metrics['secure_html_only']} domains")
        print(f"   • Network only (1pt): {metrics['secure_specific_only']} domains")
        print(f"   • Both indicators (2pts): {metrics['secure_both']} domains")
        print(f"   • Total secure: {metrics['total_secure']} domains")
        print(f"   • Weighted score: {metrics['weighted_total']} points")
        print(f"   • Possible indicators: {metrics['possible_passkey']} domains")
        print(f"   • FedCM indicators: {metrics['fedcm']} domains")
    
    print()
    print("🎯 Standardization Status:")
    print("   ✅ Uses same domain classification logic as other scrapers")
    print("   ✅ Weighted scoring system (HTML=1pt, Network=1pt, Both=2pts)")  
    print("   ✅ Domain-based evaluation (not URL-based)")
    print("   ✅ Consistent visualization style")
    print("   ✅ Two-cell notebook structure")
    print()
    print("📁 Files exported:")
    if not df.empty:
        print(f"   • Raw data: {csv_filename}")
    if domain_groups:
        print(f"   • Domain analysis: {domain_filename}")
    if 'metrics' in locals():
        print(f"   • Metrics summary: {metrics_filename}")
    if 'visualizations' in locals() and visualizations:
        print(f"   • Visualization: {viz_filename}")
    print("="*80)

except Exception as e:
    print(f"❌ Export error: {e}")
    print("Some exports may be incomplete.")